# Part 1: Simple Guide to Hyper Parameter Tuning in Neural Networks

To ensure the performane of neural networks, we cannot directly use them _out-of-the-box_.

In this tutorial, a step-by-step walkthrough on hyperparameter optimization in your neural networks is provided.

## What are we doing here?

In this tutorial, we will be optimizing a neural network and performing hyperparameter tuning in order to obtain a high-performing model on the [Beale function](https://en.wikipedia.org/wiki/Test_functions_for_optimization) — one of many test functions commonly used for studying the effectiveness of various optimization techniques.

This analysis can be reused for any function, and you should be able to try this out yourself by applying this to your course project.

Personally, I find that optimizing a neural network can be incredibly frustrating unless you have a __clear and well-defined__ procedure to follow.

## What is Beale’s Function?

Neural networks are fairly commonplace now in industry and research, but an embarrassingly large proportion of people, even experienced data scientists are unable to work with them well enough to be able to produce high-performing networks that are capable of outperforming most other algorithms.

When applied mathematicians develop a new optimization algorithm, one thing they like to do is test it on a __test function__, which is sometimes called an _artificial landscape_. These artificial landscapes help us find a way of comparing the performance of various algorithms in terms of their:
- Convergence (how fast they reach the answer)
- Precision (how close do they approximate the exact answer)
- Robustness (do they perform well for all functions or just a small subset)
- General performance (e.g. computational complexity)

From just scrolling down the Wikipedia article on optimization test functions, you can see that some of the functions are pretty nasty. Many of them have been chosen as they highlight specific issues that can plague other optimization algorithms. Thus, we select a relatively innocuous-looking function called the __Beale function__.
The Beale function looks like this:
![the Beale Function](https://miro.medium.com/max/1400/0*b6VbjuQQJVXxd_rE.jpg)

This function does not look particularly terrifying, right? The reason this is a test function is that it assesses how well the optimization algorithms perform when in flat regions with very shallow gradients. In these cases, it is particularly difficult for gradient-based optimization procedures to reach any kind of minimum, as they are unable to learn effectively. This landscape is analogous to the __loss surface__ of a neural network. When training a neural network, the goal is to find the __global minimum__ on the loss surface by performing some form of optimization — typically [_stochastic gradient descent_](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). In other words, we want to reach the central part in the Beale function surface since it has lower loss value.

In other words, there are two preliminary questions you need to answer when considering optmizing your neural networks:
- What is the __loss function__?
- How to __minimize__ the output of your loss function?

### Preliminary: A Basic Beale Function

Let's build a basic Beale function using python. To keep things simple, we assume the data only contains _two-dimensions_ (`x` and `y`). The mathematic function of Beale function is:

$$ f_{Beale}(x, y) = (1.5 - x + x \times y)^2 + (2.25 - x + (x \times y)^2)^2 + (2.625 - x + (x \times y)^3)^2 $$

Let's implement above function in Python.

In [ ]:
# define Beale's function which we want to minimize
def objective(vec): # assume `vec` is a two-dimensional vector
    x,y = vec[0], vec[1] # `x` is the first dimension of `vec` and `y` is the second
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

We then set some function boundaries since we have ballpark estimates for where the minimum is in this case (from our plot), as well as a step size for our grid mesh.

Basically, we are setting up the artifical surface in a $ 10 \times 10 $ area, and for every step in the search, we can only move a step of $1$ in the area.

In [ ]:
# function boundaries
xmin, xmax, xstep = -5, 5, 1
ymin, ymax, ystep = -5, 5, 1

We then make a mesh grid of points based on this information and are ready to find the minimum. First we can create a point as a random starting point.

In [ ]:
import numpy as np

x1, y1 = np.meshgrid(np.arange(xmin, xmax + xstep, xstep), np.arange(ymin, ymax + ystep, ystep))

Now we make a (terrible) initial guess.

In [ ]:
# initial guess
x0 = [4., 4.]
f0 = objective(x0)
print (f0) # this is the output from our Beale function

Given the value you can see that the initial value (_loss_) looks terrible.

We then use the `scipy.optimize` function and see what answer pops out.

In [ ]:
from scipy.optimize import minimize

bnds = ((xmin, xmax), (ymin, ymax)) # define the boundaries of search
minimum = minimize(objective, x0, bounds=bnds) # search the area  to minimize the function
print(minimum)

It looks like the answer is `(3, 0.5)`, and, if you plug these values into the equation you do indeed find that this is the minimum.

In [ ]:
objective([3, 0.5]) # this should equal to 0 (minimum)

## Optimization in Neural Networks
A neural network can be defined as a framework that combines inputs and tries to guess the output.

If we are lucky enough to have some results, called “the ground truth” (aka. __labels__), to compare the outputs produced by the network, we can calculate the error (difference between $\hat{y}$ and $y$). So the network guesses, calculates some error function, guesses again, trying to minimize this error, guesses again, until the error does not go down any more. This is __optimization__.

In neural networks, the most commonly used optimization algorithms are flavors of GD (gradient descent). The objective function used in gradient descent is the loss function which we want to minimize.
This tutorial will lean heavily on Keras now, so here is a brief Keras refresher.

### A Keras Refresher
Keras is a Python library for deep learning that can run on top of both Theano or TensorFlow, two powerful Python libraries for fast numerical computing created and released by Facebook and Google, respectively.

Keras was developed to make developing deep learning models as fast and easy as possible for research and practical applications. It runs on Python 2.7 or 3.5 and can seamlessly execute on GPUs and CPUs.

Keras is built on the idea of a model. At its core we have a sequence of layers called the Sequential model which is a linear stack of layers. Keras also provides the functional API, a way to define complex models, such as multi-output models, directed acyclic graphs, or models with shared layers.

We can summarize the construction of deep learning models in Keras using the Sequential model as follows:
1. Define your model: create a Sequential model and add layers.
2. Compile your model: specify loss function and optimizers and call the .compile() function.
3. Fit your model: train the model on data by calling the .fit() function.
4. Make predictions: use the model to generate predictions on new data by calling functions such as .evaluate() or .predict().

If you recall from your machine learnig knowledge with `scikit-learn`, most of the modeling also contain step 1, 3, and 4.

You may be asking yourself — how can you examine the performance of the model as it is running? This is a good question, and the answer to that is using __callbacks__.

### Callbacks: taking a peek into our model while it’s training
You can look at what is happening in various stages of your model by using `callbacks`.

A `callback` is a set of functions to be applied at given stages of the training procedure. You can use `callbacks` to get a view on any intermediate states and statistics of the model during any phase in training.

You can pass a list of `callbacks` (as the keyword argument callbacks) to the `.fit()` method of the Sequential or Model classes. The relevant methods of the callbacks will then be called at each stage of the training.

- A callback function you are already familiar with is `keras.callbacks.History()`. This is automatically included in `.fit()`.
- Another very useful one is `keras.callbacks.ModelCheckpoint` which saves the model with its weights at a certain point in the training. This can prove useful if your model is running for a __long time__ and a system failure happens. Not all is lost then. It's a good practice to save the model weights only _when an improvement is observed as measured by the `acc`, for example_.
- `keras.callbacks.EarlyStopping` stops the training when a monitored quantity has __stopped improving__.
- `keras.callbacks.LearningRateScheduler` will change the learning rate during training.

We will apply some callbacks later. For full documentation on callbacks see [here](https://keras.io/callbacks/)
.
The first thing we must do is import a lot of different functions in order to make our lives easier.


In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import datasets

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import History

from tensorflow.keras import losses
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
plt.style.use('ggplot')

print("Your tensorflow version is: ", tf.__version__)
print("Your keras version is: ", tf.keras.__version__)

Another additional step you can do if you want your network to work using random numbers but for the result to be __repeatable__ is to use a `random seed`. This basically produces the same sequence of numbers each time, although they are still _pseudorandom_ (these are a great way for comparing models and also testing for reproducibility).


In [ ]:

# fix random seed for reproducibility
np.random.seed(2020)

### Step 1 — Deciding on the network topology (not really considered optimization but is obviously very important)

We will use the MNIST dataset which consists of grayscale images of handwritten digits (0–9) whose dimension is 28x28 pixels. Each pixel is 8 bits (representing different value in a gray scale) so its value ranges from 0 to 255.

Obtaining the dataset is very easy since there is a function for it built-in to `Keras`.

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape

Our output for our X and Y data is `(60000, 28, 28)` and `(60000,1)` respectively.

It is always a good suggestion to print some of the data to check the values (and the data type if necessary).


We can check the training data by looking at one image of each of the digits to make sure that none of them are missing from our data.


In [ ]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])

The last check is for the dimensions of the training and test sets, which can be done relatively easily:

In [ ]:

print(f'We have {x_train.shape[0]} train samples')
print(f'We have {x_test.shape[0]} test samples')

### Preprocessing the data
To run our NN we need to preprocess the data (these steps can be performed interchangeably):

1. We need to make the 2D image arrays into 1D (__flatten__ them). We can either perform this by using array reshaping with `numpy.reshape()` or the keras' method for this: a layer called `keras.layers.Flatten` which transforms the format of the images from a 2d-array (of $28 \times 28$ pixels), to a 1D-array of $28 \times 28 = 784$ pixels.
2. We need to normalize the pixel values (give them values between 0 and 1) using the following transformation:
$$ x' = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In our case, the minimum is 0 and the maximum is 255, so the formula becomes simply $𝑥'=𝑥/255$.

In [ ]:
# normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# reshape the data into 1D vectors
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

num_classes = 10

# Check the column length
x_train.shape[1]

In [ ]:
x_train[0]

We can see now our features are in 784-element vectors, and the values are between `[0, 1]`.

We now want to one-hot encode our target (label).

In [ ]:
# Convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

y_train[:2]

Now we have prepared the data for training our model.

### Step 2 — Adjusting the `learning rate`

One of the most common optimization algorithms is __Stochastic Gradient Descent__ (SGD). The hyperparameters that can be optimized in SGD are `learning rate`, `momentum`, `decay` and `nesterov`.

- `Learning rate` controls the weight at the end of each batch, and
- `momentum` controls how much to let the previous update influence the current weight update
- `Decay` indicates the learning rate decay over each update, and
- `nesterov` takes the value “True” or “False” depending on if we want to apply Nesterov momentum.

Typical values for those hyperparameters are `lr=0.01, decay=1e-6, momentum=0.9`, and `nesterov=True`.

The learning rate hyperparameter goes into the optimizer function which we will see below. Keras has a default learning rate scheduler in the `SGD` optimizer that decreases the learning rate during the stochastic gradient descent optimization algorithm. The learning rate is decreased according to this formula:
$$ lr=lr \times 1/(1+decay \times epoch) $$

Selecting a good learning rate would definitely impact your model performance. See image below.
![learning rates and model performance](https://miro.medium.com/max/1400/1*Dbr6vHLUeUk-fzDf9VWbUw.png)

Let’s implement a learning rate adaptation schedule in `Keras`. We'll start with `SGD` and a learning rate value of `0.1`. We will then train the model for `60` epochs and set the decay argument to `0.0016` (`0.1/60`). We also include a momentum value of `0.8` since that seems to work well when using an adaptive learning rate.

The reason behind the declining learning rate is that after each round of training, we want the search to be more specific. It is analoguous to tuning the radio seeking a station - at first you can turn the knob fast, then when you get closer you want to turn the knob more carefully in case you missed the station.

In [ ]:
epochs=60
learning_rate = 0.1
decay_rate = learning_rate / epochs # 0.016
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

Next, we build the architecture of the neural network, for illustration purposes, we will build a simple MLP network:

In [ ]:
# build the model
input_dim = x_train.shape[1]

lr_model = Sequential()
lr_model.add(Dense(64, activation=tf.nn.relu, kernel_initializer='uniform',
                input_dim = input_dim))
lr_model.add(Dropout(0.1))
lr_model.add(Dense(64, kernel_initializer='uniform', activation=tf.nn.relu))
lr_model.add(Dense(num_classes, kernel_initializer='uniform', activation=tf.nn.softmax))

# compile the model
lr_model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['acc'])

We can now run the model and see how well it performs. This took ~20 minutes on Colab and may be faster or slower on yours depending on what machine you are using.


In [ ]:
%%time
# Fit the model
batch_size = int(input_dim/100)

lr_model_history = lr_model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

After it has finished running, we can plot the accuracy and loss function as a function of epochs for the training and test sets to see how the network performed.

In [ ]:
# Plot the loss function
print("loss function over training")
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(lr_model_history.history['loss']), 'r', label='train')
ax.plot(np.sqrt(lr_model_history.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Loss', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20);

# Plot the accuracy
print("accuracy over training")
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(lr_model_history.history['acc']), 'r', label='train')
ax.plot(np.sqrt(lr_model_history.history['val_acc']), 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Accuracy', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20);

We can see from above that we did not overfit the model.

We will now look at applying a customized learning rate.

### Apply a custom learning rate change using LearningRateScheduler

Write a function that performs the exponential learning rate decay as indicated by the following formula:

$$ 𝑙𝑟=𝑙𝑟_{0} \times 𝑒^{−𝑘𝑡} $$
In which, $𝑙𝑟_{0}$ is the learning rate of the previous epoch, $k$ is the decay rate, and $t$ is the number of epochs.

This is very similar to before so I will do this in one code block and describe the differences.

In [ ]:
epochs = 60
learning_rate = 0.1 # initial learning rate
decay_rate = 0.1
momentum = 0.8

# define the optimizer function
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

input_dim = x_train.shape[1]
num_classes = 10
batch_size = 196

# build the model
exponential_decay_model = Sequential()
exponential_decay_model.add(Dense(64, activation=tf.nn.relu, kernel_initializer='uniform', input_dim = input_dim))
exponential_decay_model.add(Dropout(0.1))
exponential_decay_model.add(Dense(64, kernel_initializer='uniform', activation=tf.nn.relu))
exponential_decay_model.add(Dense(num_classes, kernel_initializer='uniform', activation=tf.nn.softmax))

# compile the model
exponential_decay_model.compile(loss='categorical_crossentropy',
                                optimizer=sgd,
                                metrics=['acc'])

# define the learning rate change
def exp_decay(epoch):
    lrate = learning_rate * np.exp(-decay_rate*epoch)
    return lrate

# learning schedule callback
loss_history = History()
lr_rate = LearningRateScheduler(exp_decay)
callbacks_list = [loss_history, lr_rate]

# you invoke the LearningRateScheduler during the .fit() phase
exponential_decay_model_history = exponential_decay_model.fit(x_train, y_train,
                                    batch_size=batch_size,
                                    epochs=epochs,
                                    callbacks=callbacks_list,
                                    verbose=1,
                                    validation_data=(x_test, y_test))

We see here that the only thing that has changed here is the presence of the `exp_decay` function that we defined, and its use in the `LearningRateScheduler` function. Notice we also chose to add them as the callbacks to our model this time.

We can now plot the learning rate and loss functions as functions of the number of epochs. The learning rate plot is incredibly smooth as it follows our predefined exponentially decaying function.

In [ ]:
# Plot the loss function
print("(above) loss function over training")
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(exponential_decay_model_history.history['loss']), 'r', label='train')
ax.plot(np.sqrt(exponential_decay_model_history.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Loss', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20);

# Plot the accuracy
print("(below) accuracy over training")
fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(exponential_decay_model_history.history['acc']), 'r', label='train')
ax.plot(np.sqrt(exponential_decay_model_history.history['val_acc']), 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Accuracy', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20);

You can observe that both the accuracy curve and the loss curve are smoother now as compared to before.

This shows you that developing a learning rate scheduler can be a helpful way to improve neural network performance.

### Step 3 — Choosing an `optimizer` and a `loss function`
When constructing a model and using it to make our predictions, for example, to assign label scores to images (“cat”, “plane”, etc), we want to measure our success or failure by defining a “loss” function (or objective function). The goal of optimization is to efficiently calculate the parameters/weights that __minimize this loss function__. `keras` provides various types of loss functions.

Sometimes the “loss” function measures the “distance”. We can define this “distance” between two data points in various ways suitable to the problem or dataset. The distance used depends on the data type and the specific problem that is being tackled. For example, in natural language processing (which analyses textual data), the _Hamming distance_ is much more common to use.

#### Distance
- Euclidean
- Manhattan
- others such as Hamming which measures distances between strings, for example. The Hamming distance of “carolin” and “cathrin” is 3.

#### Loss functions
- MSE/RMSE (for regression)
- categorical cross-entropy (for classification)
- binary cross entropy (for (binary) classification)


In [ ]:
# build the model
input_dim = x_train.shape[1]

model = Sequential()
model.add(Dense(64, activation=tf.nn.relu, kernel_initializer='uniform',
                input_dim = input_dim)) # fully-connected layer with 64 hidden units
model.add(Dropout(0.1))
model.add(Dense(64, kernel_initializer='uniform', activation=tf.nn.relu))
model.add(Dense(num_classes, kernel_initializer='uniform', activation=tf.nn.softmax))

# defining the parameters for RMSprop (I used the keras defaults here)
rms = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer=rms,
              metrics=['acc'])

### Step 4 — Deciding on the batch size and number of epochs
The __batch size__ defines the number of samples that will be propagated through the network, in __one iteration__.

For instance, let’s say you have 1000 training samples and you want to set up a `batch_size` equal to 100. The algorithm takes the _first_ 100 samples (from $1^{st}$ to $100^{th}$) from the training dataset and trains the network. Next, it takes the _second_ 100 samples (from $101^{st}$ to $200^{th}$) and trains the network again. We can keep doing this procedure until we have propagated all samples through the network (10 iterations).

Advantages of using a smaller batch size:
- It requires __less memory__. Since you train the network using fewer samples, the overall training procedure requires less memory. That’s especially important if you are not able to fit the whole dataset in your machine’s memory.
- Typically networks __train faster__ with mini-batches. That’s because we update the weights after each propagation.
Disadvantages of using a smaller batch size:
- The smaller the batch, the less accurate the estimate of the gradient will be.

The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset.

One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters (e.g. aforementioned 100 iterations consist one epoch). An epoch is comprised of one or more batches.

There are no hard and fast rules for selecting batch sizes or the number of epochs, and there is no guarantee that increasing the number of epochs provides a better result than a lesser number.

In [ ]:
%%time
batch_size = input_dim
epochs = 60

model_history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(model_history.history['acc']), 'r', label='train_acc')
ax.plot(np.sqrt(model_history.history['val_acc']), 'b' ,label='val_acc')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Accuracy', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20)

fig, ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(np.sqrt(model_history.history['loss']), 'r', label='train')
ax.plot(np.sqrt(model_history.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch', fontsize=20)
ax.set_ylabel(r'Loss', fontsize=20)
ax.legend()
ax.tick_params(labelsize=20)

You can see from above, as we increase the `batch_size` while holding the number of epochs steady, we reduce the training time from ~20 minutes to ~22 seconds.

But you can also observe that both the accuracy and loss curves are "bumpier" - meaning the performance of the model has __declined__.